In [3]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [4]:
# Input Data Set #
Data ="Iris"
JobNameAbbrev = "IS"

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,100)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0,1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "rashomon_bound_adder": [0.005, 0.007, 0.009, 0.011, 0.013, 0.037, 0.039, 0.041, 0.043, 0.045],
                       "Type": ["Classification"]}

# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Include/exclude Random Forest Simulations

In [5]:
# ### Include Random Forest ###
# RandomForestParameterDictionary = {"Data":[Data],
#                        "Seed":list(range(0,100)),
#                        "TestProportion":[0.2],
#                        "CandidateProportion":[0.8],
#                        "SelectorType":["TreeEnsembleQBCFunction"],
#                        "ModelType":["RandomForestClassificationFunction"],
#                        "UniqueErrorsInput": [0],
#                        "n_estimators": [100], 
#                        "regularization": [0.00],
#                        "rashomon_bound_adder": [0],
#                        "Type": ["Classification"]}
# RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# # NOTE: Comment out chunk to not include random forest simulations. ###
# ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
# ParameterVector = ParameterVector.sort_values("Seed")
# ParameterVector.index = range(0, ParameterVector.shape[0])

# Job and Output Name

In [6]:
# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["rashomon_bound_adder"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_DPL", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_UNQ", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True))

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

# Save Parameter Vector

In [7]:
# Save 
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)

In [8]:
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,rashomon_bound_adder,Type,JobName,Output
0,Iris,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.005,Classification,0IS_DPL0.005,Iris/TreeFarms/Raw/0IS_DPL0.005.pkl
1,Iris,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.007,Classification,0IS_DPL0.007,Iris/TreeFarms/Raw/0IS_DPL0.007.pkl
2,Iris,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.009,Classification,0IS_DPL0.009,Iris/TreeFarms/Raw/0IS_DPL0.009.pkl
3,Iris,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.011,Classification,0IS_DPL0.011,Iris/TreeFarms/Raw/0IS_DPL0.011.pkl
4,Iris,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.013,Classification,0IS_DPL0.013,Iris/TreeFarms/Raw/0IS_DPL0.013.pkl
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Iris,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.037,Classification,99IS_UNQ0.037,Iris/TreeFarms/Raw/99IS_UNQ0.037.pkl
1996,Iris,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.039,Classification,99IS_UNQ0.039,Iris/TreeFarms/Raw/99IS_UNQ0.039.pkl
1997,Iris,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.041,Classification,99IS_UNQ0.041,Iris/TreeFarms/Raw/99IS_UNQ0.041.pkl
1998,Iris,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.043,Classification,99IS_UNQ0.043,Iris/TreeFarms/Raw/99IS_UNQ0.043.pkl
